In [ ]:
#Conectar a drive

from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
#Extracción de caracteristicas de una imagen (Sacar RGB en un arreglo numpy)

from PIL import Image
import numpy
def get_image(image_path):
    image = Image.open(image_path, "r")
    width, height = image.size
    pixel_values = list(image.getdata())
    if image.mode == "RGB":
        channels = 3
    elif image.mode == "L":
        channels = 1
    else:
        print("Unknown mode: %s" % image.mode)
        return None
    pixel_values = numpy.array(pixel_values).reshape((width, height, channels))
    return pixel_values

In [ ]:
#Sacar el meta dato de todas las imagenes en la carpeta (130 elementos)
import os
your_path = r'C:\Users\subce\Downloads\Imagenes_numeradas'
imagenes = []
files = os.listdir(your_path)
imagenes_path = []
current = 0
for file in files:
    if os.path.isfile(os.path.join(your_path, file)):
        f = open(os.path.join(your_path, file),'r', encoding = "ISO-8859-1")
        imagenes.append(get_image(os.path.join(your_path, file)))
        imagenes_path.append(os.path.join(your_path, file))
        print(current)
        current = current + 1
        f.close()

In [ ]:
#Codigo para ya tener el data completa diviendo cada imagen en 25 cuadrantes, siendo un total de 3250 elementos
#Por las limitaciones de el equipo, se tuvo que dividir esta parte en 3 ejecuciones, debido a que si se dejaba correr en una sola ejecución la maquina se congelaba totalmente. 

data = []
cuadrante = []

for x in range(0,10):
  cuadrante = []
  if (x == 1 or x == 127 or x == 130):
    for p in range(0, 25):
        data.append([])   
    continue;
  if (imagenes[x] is None):
    for p in range(0, 25):
        data.append([])   
    continue
  print(x)
  h = len(imagenes[x])
  w = len(imagenes[x][0])
  division_w = int(w / 5)
  actual_w = 0
  actual_h = 0
  contador = 0
  division_h = int(h / 5)
  for y in range(0, 5): #Nos movemos en Vertical por cuadrante
    actual_w = 0  
    for z in range(0, 5): #Nos movemos en horizontal por cuadrante
      cuadrante = []
      for a in range(actual_h, actual_h + division_h): #Nos movemos en vertical 
        for b in range(actual_w, actual_w + division_w):#Nos movemos en horizontal
          cuadrante.append(imagenes[x][a][b])  
      actual_w = actual_w + division_w
      data.append(cuadrante)
    actual_h = actual_h + division_h
imagenes = []
      
    
        






In [ ]:
#Ejemplos de como vienen distribuidos los datos en data, [R, G, B]
print((data[0][1]))
print((data[3][1]))

In [ ]:
#Obtener path imagenes. 
import os
your_path = '/content/gdrive/MyDrive/Imagenes incendios forestales'
imagenes = []
files = os.listdir(your_path)
imagenes_path = []
current = 0
for file in files:
    if os.path.isfile(os.path.join(your_path, file)):
        imagenes_path.append(os.path.join(your_path, file))
        print(current)
        current = current + 1

In [ ]:
#Algoritmo usado para dibujar una matriz de 5x5 en cada imagen, las imagenes se guardan localmente para despues clasificarlas. 
import math
from PIL import Image, ImageDraw
step_count = 5 #Numero de cuadrados
  
for z in range(0,130):
  img = Image.open(imagenes_path[z], "r")
  w, h = img.size

  draw = ImageDraw.Draw(img)
  y_start = 0
  y_end = h
  step_size = int(w / step_count)
  for x in range(0, w, step_size):
    line = ((x, y_start), (x, y_end))
    draw.line(line, fill=128)
  x_start = 0
  x_end = w

  step_size = int(h / step_count)
  for y in range(0, h, step_size):
    line = ((x_start, y), (x_end, y))
    draw.line(line, fill=128)
  del draw
  print(z)
  img.save('/content/gdrive/MyDrive/Imagenes_numeradas/'+str(z)+'.png')




In [ ]:
#Algoritmo usado para la clasificacion de las imagenes, visualmente en la Notebook como Jupyter es imposible crear una ventana, por lo que se decidió ver las imagenes localmente y clasificarlas aqui. 
from IPython.display import clear_output 
cuadricula = []
imgs_clasificacion = []

for x in range(0,25):
    cuadricula.append("fuego")
mayoria = "fuego"

def cambiar_may(cuadricula, categoria):
  for x in range(0,25):
    cuadricula[x] = categoria
  return cuadricula

for x in range(0,70):
  cambiar_may(cuadricula, "fuego")
  inc = ""
  humo = ""
  excepcion = ""
  no_inc = ""
  no_inc = input("Coloca el numero de la región que pertenece a no incendio")
  if no_inc == "no":
    cambiar_may(cuadricula,"no")
    inc = input("Coloca el numero de la región que pertenece a  incendio")
    humo   = input("Coloca el numero de la región que pertenece a humo")
    excepcion   = input("Coloca el numero de la región que no se incluirá")
  elif no_inc == "humo":
    cambiar_may(cuadricula,"humo")
    no_inc = input("Coloca el numero de la región que pertenece a no incendio")
    inc = input("Coloca el numero de la región que pertenece a  incendio")
    excepcion   = input("Coloca el numero de la región que no se incluirá")
  else:
    humo   = input("Coloca el numero de la región que pertenece a humo")
    excepcion   = input("Coloca el numero de la región que no se incluirá")

  no_inc    = no_inc.split(",")
  humo      = humo.split(",")
  excepcion = excepcion.split(",")

  if(isinstance(inc, str)):
    inc       = inc.split(",")

  for x in no_inc:
    if x == '' or x == 'no':
      break
    cuadricula[int(x) - 1] = "no"
  for x in humo:
    if x == '':
      break
    cuadricula[int(x) - 1] = "humo"
  for x in excepcion:
    if x == '':
      break
    cuadricula[int(x) - 1] = "excepcion"
  for x in inc:
    if x == '':
      break
    cuadricula[int(x) - 1] = "fuego"
  
  imgs_clasificacion.extend(cuadricula)
  clear_output()




In [ ]:
#Este algoritmo cumplió con la funcionalidad de almacenar la clasificación de una forma segura, debido a que fue un proceso tardado. 
primeras_60 = [['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'no', 'no', 'fuego', 'fuego', 'no', 'no', 'no',
       'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no',
       'no', 'no'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no',
       'no', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no', 'no'],
       ['no', 'no', 'humo', 'humo', 'humo', 'no', 'humo', 'humo', 'humo',
       'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'], ['no', 'fuego', 'fuego', 'fuego', 'humo', 'no', 'fuego', 'fuego',
       'fuego', 'humo', 'no', 'fuego', 'fuego', 'fuego', 'humo', 'no',
       'fuego', 'fuego', 'fuego', 'humo', 'no', 'fuego', 'fuego', 'fuego',
       'humo'], ['humo', 'no', 'humo', 'humo', 'humo', 'humo', 'no', 'humo', 'humo',
       'humo', 'fuego', 'fuego', 'fuego', 'humo', 'humo', 'humo', 'humo',
       'fuego', 'fuego', 'fuego', 'humo', 'humo', 'humo', 'humo', 'humo'],
       ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'humo',
       'humo', 'humo', 'fuego', 'no', 'humo', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no'], ['no', 'no', 'no', 'no', 'no', 'no', 'fuego', 'fuego', 'no', 'no',
       'no', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no'],
       ['no', 'humo', 'humo', 'no', 'no', 'no', 'fuego', 'humo', 'no',
       'no', 'no', 'no', 'no', 'humo', 'no', 'excepcion', 'excepcion',
       'excepcion', 'excepcion', 'excepcion', 'excepcion', 'excepcion',
       'excepcion', 'excepcion', 'excepcion'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no',
       'humo', 'no'], ['no', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego'], ['no', 'no', 'no', 'no', 'no', 'fuego', 'fuego', 'humo',
       'excepcion', 'no', 'no', 'fuego', 'fuego', 'no', 'no', 'no', 'no',
       'fuego', 'no', 'no', 'no', 'no', 'no', 'no', 'no'], ['excepcion', 'humo', 'humo', 'humo', 'humo', 'excepcion', 'humo',
       'humo', 'humo', 'humo', 'excepcion', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego',
       'fuego', 'humo', 'humo'], ['no', 'no', 'humo', 'no', 'no', 'no', 'no', 'humo', 'no', 'no',
       'no', 'no', 'humo', 'no', 'no', 'no', 'no', 'humo', 'no', 'no',
       'no', 'no', 'fuego', 'no', 'no'], ['fuego', 'fuego', 'humo', 'no', 'no', 'fuego', 'fuego', 'humo',
       'humo', 'no', 'fuego', 'fuego', 'humo', 'humo', 'humo', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'no'], ['no', 'fuego', 'humo', 'humo', 'humo', 'no', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'humo',
       'humo', 'fuego', 'no', 'no', 'fuego', 'no', 'no', 'no', 'no'],
       ['humo', 'humo', 'humo', 'fuego', 'fuego', 'humo', 'humo', 'humo',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no',
       'no', 'no'], ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'humo', 'no', 'no', 'no',
       'fuego', 'humo', 'no', 'no', 'no', 'no', 'fuego', 'no', 'no', 'no',
       'no', 'no', 'no', 'no'], ['fuego', 'fuego', 'fuego', 'fuego', 'humo', 'fuego', 'fuego',
       'fuego', 'fuego', 'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'no',
       'excepcion', 'excepcion', 'excepcion', 'no', 'no', 'fuego',
       'fuego', 'fuego', 'no', 'no'], ['humo', 'fuego', 'fuego', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'excepcion',
       'excepcion', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'fuego', 'humo', 'humo', 'humo',
       'fuego', 'humo', 'humo', 'humo', 'humo', 'no', 'no', 'humo', 'no'],
       ['no', 'humo', 'humo', 'humo', 'humo', 'no', 'humo', 'humo', 'humo',
       'humo', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego',
       'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no'],
       ['humo', 'humo', 'humo', 'humo', 'no', 'humo', 'humo', 'no', 'humo',
       'humo', 'no', 'fuego', 'no', 'fuego', 'no', 'no', 'no',
       'excepcion', 'excepcion', 'no', 'no', 'no', 'no', 'no', 'no'],
       ['no', 'fuego', 'humo', 'humo', 'fuego', 'no', 'humo', 'humo',
       'fuego', 'fuego', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no',
       'fuego', 'excepcion', 'excepcion', 'fuego', 'no', 'no',
       'excepcion', 'excepcion', 'excepcion'], ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'fuego',
       'fuego', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'fuego', 'humo', 'humo', 'humo', 'humo', 'fuego',
       'fuego', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'no', 'no',
       'no'], ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'no', 'no', 'fuego',
       'fuego', 'no', 'no', 'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'no', 'fuego',
       'fuego', 'fuego', 'humo', 'no', 'no', 'no', 'no', 'no', 'no'],
       ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego'], ['fuego', 'fuego', 'fuego', 'fuego', 'excepcion', 'humo', 'humo',
       'fuego', 'fuego', 'excepcion', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'fuego', 'fuego', 'no', 'no', 'fuego', 'no', 'no',
       'no'], ['humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'humo', 'humo',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'no'], ['no', 'humo', 'humo', 'no', 'no', 'no', 'no', 'humo', 'fuego',
       'fuego', 'no', 'no', 'fuego', 'fuego', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no'], ['no', 'humo', 'humo', 'humo', 'no', 'no', 'humo', 'humo', 'humo',
       'no', 'no', 'humo', 'humo', 'humo', 'humo', 'no', 'humo', 'humo',
       'humo', 'no', 'no', 'humo', 'fuego', 'fuego', 'no'], ['no', 'no', 'no', 'no', 'no', 'humo', 'humo', 'humo', 'humo',
       'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'fuego', 'no'], ['no', 'fuego', 'fuego', 'fuego', 'no', 'no', 'fuego', 'fuego',
       'no', 'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'humo', 'fuego',
       'fuego', 'fuego', 'fuego', 'humo', 'no', 'fuego', 'fuego', 'fuego',
       'no'], ['fuego', 'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'humo',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no',
       'fuego', 'fuego', 'fuego'], ['fuego', 'humo', 'humo', 'fuego', 'fuego', 'fuego', 'fuego',
       'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no',
       'no', 'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'fuego', 'humo', 'humo', 'humo', 'humo', 'fuego', 'no', 'humo',
       'humo', 'humo', 'fuego', 'no', 'humo', 'humo', 'humo', 'humo',
       'no'], ['no', 'fuego', 'humo', 'humo', 'humo', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego'], ['no', 'no', 'no', 'no', 'no', 'no', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'fuego', 'fuego', 'fuego', 'no', 'no', 'fuego',
       'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no', 'no'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'no', 'fuego'], ['fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'no', 'fuego', 'fuego', 'fuego', 'no', 'no', 'fuego',
       'fuego', 'fuego', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'humo',
       'humo', 'humo', 'fuego', 'fuego', 'fuego', 'humo', 'fuego', 'no',
       'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no'],
       ['humo', 'fuego', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego',
       'fuego', 'humo', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no',
       'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'fuego', 'fuego',
       'no'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego'], ['fuego', 'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'humo',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'fuego', 'fuego', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'no',
       'humo', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no'],
       ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'fuego',
       'fuego', 'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'humo', 'no',
       'no', 'humo', 'humo', 'humo', 'no', 'no', 'no', 'no', 'humo'],
       ['humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'fuego',
       'fuego', 'fuego', 'no'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'fuego', 'fuego', 'fuego', 'no', 'no', 'fuego', 'fuego',
       'fuego', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'fuego', 'no', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'fuego'],
       ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego',
       'fuego', 'fuego', 'fuego', 'humo', 'no', 'no', 'no', 'no', 'no'],
       ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego',
       'fuego', 'fuego', 'fuego', 'humo', 'fuego', 'fuego', 'fuego',
       'humo', 'humo', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'no',
       'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'no', 'humo',
       'humo', 'fuego', 'fuego', 'no', 'fuego', 'fuego', 'fuego', 'fuego',
       'no'], ['humo', 'humo', 'no', 'humo', 'fuego', 'humo', 'humo', 'no',
       'humo', 'fuego', 'humo', 'humo', 'no', 'humo', 'humo', 'fuego',
       'humo', 'no', 'fuego', 'fuego', 'humo', 'humo', 'no', 'fuego',
       'fuego']]

ultimos_70 = [['humo', 'humo', 'humo', 'humo', 'fuego', 'humo', 'humo', 'humo',
       'fuego', 'fuego', 'humo', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'no'], ['no', 'humo', 'humo', 'humo', 'no', 'no', 'no', 'fuego', 'fuego',
       'fuego', 'no', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'no', 'no', 'no', 'excepcion', 'excepcion', 'excepcion',
       'excepcion', 'excepcion'], ['no', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no',
       'no'], ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'fuego', 'no', 'no', 'no', 'fuego', 'fuego', 'no', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'fuego', 'no', 'no', 'no', 'no', 'no', 'no',
       'fuego', 'fuego', 'no', 'no', 'no', 'no', 'fuego', 'no'],
       ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'fuego', 'no', 'no',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego',
       'fuego', 'fuego', 'fuego', 'no', 'no', 'fuego', 'fuego', 'fuego'],
       ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'fuego', 'humo', 'humo', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'humo', 'no',
       'no', 'no', 'no', 'fuego', 'no', 'no', 'no', 'no', 'no'],
       ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'humo', 'no',
       'fuego', 'fuego', 'fuego', 'humo', 'no', 'no', 'no', 'no', 'no'],
       ['no', 'no', 'fuego', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'fuego', 'fuego'],
       ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'fuego', 'no', 'humo', 'fuego', 'humo', 'no', 'no',
       'no', 'no', 'humo', 'no', 'no', 'no', 'no', 'no'], ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'fuego', 'humo', 'humo',
       'no', 'no', 'fuego', 'humo', 'humo', 'humo', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego'],
       ['no', 'humo', 'humo', 'humo', 'no', 'no', 'humo', 'humo', 'humo',
       'no', 'no', 'fuego', 'no', 'no', 'no', 'no', 'fuego', 'fuego',
       'fuego', 'no', 'no', 'no', 'no', 'no', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'no', 'no', 'no', 'no', 'no'],
       ['no', 'no', 'no', 'fuego', 'fuego', 'fuego', 'no', 'fuego',
       'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'],
       ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no',
       'no', 'no'], ['no', 'no', 'no', 'no', 'no', 'no', 'fuego', 'fuego', 'humo', 'no',
       'no', 'fuego', 'fuego', 'humo', 'no', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego'],
       ['humo', 'humo', 'fuego', 'humo', 'humo', 'humo', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no',
       'no', 'fuego'], ['no', 'no', 'fuego', 'fuego', 'humo', 'no', 'no', 'no', 'fuego',
       'humo', 'no', 'no', 'no', 'fuego', 'fuego', 'no', 'no', 'no',
       'fuego', 'fuego', 'no', 'no', 'no', 'fuego', 'no'], ['fuego', 'fuego', 'humo', 'humo', 'humo', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego', 'fuego',
       'fuego', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no'], ['humo', 'humo', 'no', 'no', 'no', 'humo', 'humo', 'humo', 'no',
       'no', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'no', 'no', 'no', 'no', 'no'], ['no', 'humo', 'humo', 'humo', 'humo', 'no', 'humo', 'humo', 'humo',
       'humo', 'no', 'humo', 'humo', 'humo', 'humo', 'no', 'fuego',
       'fuego', 'humo', 'no', 'no', 'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'humo',
       'fuego', 'fuego', 'humo', 'humo', 'fuego', 'fuego', 'fuego', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'], ['fuego', 'fuego', 'fuego', 'humo', 'humo', 'fuego', 'fuego',
       'fuego', 'humo', 'humo', 'fuego', 'fuego', 'fuego', 'humo', 'humo',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no',
       'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego',
       'humo', 'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego'], ['no', 'no', 'no', 'humo', 'humo', 'no', 'no', 'no', 'humo', 'humo',
       'no', 'no', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego'],
       ['no', 'no', 'fuego', 'humo', 'no', 'no', 'no', 'fuego', 'humo',
       'no', 'no', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego', 'fuego', 'no'],
       ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego',
       'fuego', 'fuego', 'no'], ['fuego', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'humo', 'humo',
       'fuego', 'fuego', 'fuego', 'fuego', 'humo', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'no', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'], ['fuego', 'fuego', 'humo', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'fuego', 'no',
       'excepcion', 'excepcion', 'excepcion', 'excepcion', 'excepcion',
       'excepcion', 'excepcion', 'excepcion', 'excepcion', 'excepcion'],
       ['fuego', 'no', 'humo', 'humo', 'fuego', 'humo', 'fuego', 'humo',
       'no', 'no', 'no', 'fuego', 'fuego', 'no', 'fuego', 'no', 'no',
       'no', 'no', 'fuego', 'no', 'excepcion', 'excepcion', 'fuego', 'no'],
       ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'humo', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'],
       ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'fuego', 'fuego', 'fuego', 'no', 'no', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'fuego', 'fuego', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'no'], ['no', 'no', 'humo', 'humo', 'no', 'no', 'no', 'fuego', 'fuego',
       'fuego', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'excepcion',
       'excepcion', 'fuego', 'fuego', 'fuego', 'excepcion', 'excepcion',
       'excepcion', 'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'fuego', 'humo', 'humo', 'humo', 'humo', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no'],
       ['no', 'no', 'no', 'humo', 'humo', 'no', 'no', 'no', 'humo', 'humo',
       'no', 'fuego', 'fuego', 'humo', 'humo', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'no', 'no', 'no', 'no', 'fuego'], ['fuego', 'humo', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'humo',
       'humo', 'fuego', 'no', 'fuego', 'fuego', 'humo', 'fuego', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'excepcion',
       'excepcion', 'excepcion', 'excepcion', 'excepcion'], ['humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'humo', 'humo',
       'humo', 'humo', 'fuego', 'fuego', 'fuego', 'humo', 'humo', 'fuego',
       'fuego', 'fuego', 'humo', 'humo', 'fuego', 'fuego', 'humo', 'humo',
       'humo'], ['no', 'fuego', 'humo', 'humo', 'humo', 'fuego', 'fuego', 'humo',
       'humo', 'humo', 'fuego', 'fuego', 'humo', 'humo', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no'],
       ['humo', 'humo', 'no', 'no', 'no', 'humo', 'humo', 'no', 'no', 'no',
       'humo', 'humo', 'humo', 'fuego', 'fuego', 'humo', 'humo', 'fuego',
       'fuego', 'no', 'no', 'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'fuego', 'fuego', 'humo', 'fuego', 'fuego',
       'fuego', 'no', 'fuego', 'fuego', 'humo', 'humo', 'humo', 'no',
       'excepcion', 'excepcion', 'excepcion', 'excepcion', 'no', 'no',
       'no', 'no', 'no'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego'], ['no', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego',
       'humo', 'humo', 'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'humo',
       'no'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'humo', 'fuego', 'fuego', 'no', 'no', 'no', 'no',
       'fuego', 'fuego'], ['no', 'no', 'no', 'no', 'no', 'fuego', 'no', 'no', 'no', 'no',
       'fuego', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'no',
       'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'no', 'no', 'no', 'no', 'humo',
       'humo', 'fuego', 'no', 'no', 'no', 'no', 'fuego', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'fuego', 'no'], ['no', 'humo', 'humo', 'humo', 'humo', 'no', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'fuego', 'humo', 'humo', 'humo',
       'humo', 'fuego', 'fuego', 'humo', 'fuego', 'fuego', 'fuego',
       'fuego'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no',
       'fuego', 'fuego', 'fuego', 'no'], ['no', 'no', 'no', 'humo', 'humo', 'no', 'fuego', 'fuego', 'humo',
       'humo', 'no', 'fuego', 'fuego', 'humo', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'fuego', 'no', 'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'fuego', 'humo', 'fuego', 'fuego',
       'fuego', 'no', 'humo', 'humo', 'fuego', 'fuego', 'no', 'no', 'no',
       'no', 'fuego', 'no', 'no', 'no', 'no', 'no', 'no'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego'], ['fuego', 'fuego', 'fuego', 'humo', 'no', 'fuego', 'fuego', 'fuego',
       'humo', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'no',
       'no', 'no', 'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no'],
       ['fuego', 'fuego', 'fuego', 'fuego', 'humo', 'fuego', 'fuego',
       'fuego', 'fuego', 'humo', 'fuego', 'fuego', 'fuego', 'fuego',
       'humo', 'fuego', 'fuego', 'fuego', 'fuego', 'humo', 'fuego',
       'fuego', 'fuego', 'fuego', 'humo'], ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'no', 'no', 'fuego', 'fuego', 'fuego',
       'no', 'no', 'fuego', 'fuego', 'fuego', 'no', 'no', 'no', 'no',
       'no'], ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego'], ['humo', 'fuego', 'humo', 'fuego', 'humo', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'humo', 'fuego', 'no', 'no',
       'no', 'fuego', 'no'], ['no', 'fuego', 'fuego', 'no', 'no', 'no', 'fuego', 'fuego', 'no',
       'no', 'humo', 'fuego', 'fuego', 'no', 'no', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'no', 'no', 'fuego', 'fuego', 'no'], ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'no', 'fuego', 'fuego', 'fuego', 'fuego', 'no', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'humo',
       'humo', 'humo', 'no', 'no', 'no', 'no', 'no', 'no', 'fuego',
       'fuego', 'fuego', 'no', 'no', 'no', 'no', 'no', 'no'], ['humo', 'humo', 'humo', 'humo', 'humo', 'humo', 'fuego', 'humo',
       'humo', 'humo', 'humo', 'fuego', 'fuego', 'no', 'no', 'no',
       'fuego', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no'],
       ['fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego', 'fuego',
       'fuego', 'fuego', 'fuego', 'fuego']]

clasificacion = primeras_60 + ultimos_70
clasificacion_1 = []
for x in clasificacion:
    for z in x:
        clasificacion_1.append(z)

In [ ]:
#Utilizado para contar los elementos en nuestra clasificación. 
from collections import Counter
from itertools import chain

datos = primeras_60 + ultimos_70
contador = Counter()
#datos = " ".join(chain.from_iterable(datos))

print(len(datos))
for x in range(0,130):
    contador.update(datos[x])
    
print(contador)

In [ ]:
#Algoritmo para convertir RGB a SIEL*a*b
def rgb2lab2(inputColor):

    num = 0
    RGB = [0, 0, 0]

    for value in inputColor:
        value = float(value) / 255

        if value > 0.04045:
            value = ((value + 0.055) / 1.055) ** 2.4
        else:
            value = value / 12.92

        RGB[num] = value * 100
        num = num + 1

    XYZ = [0, 0, 0, ]

    X = RGB[0] * 0.4124 + RGB[1] * 0.3576 + RGB[2] * 0.1805
    Y = RGB[0] * 0.2126 + RGB[1] * 0.7152 + RGB[2] * 0.0722
    Z = RGB[0] * 0.0193 + RGB[1] * 0.1192 + RGB[2] * 0.9505
    XYZ[0] = round(X, 4)
    XYZ[1] = round(Y, 4)
    XYZ[2] = round(Z, 4)

    # Observer= 2°, Illuminant= D65
    XYZ[0] = float(XYZ[0]) / 95.047         # ref_X =  95.047
    XYZ[1] = float(XYZ[1]) / 100.0          # ref_Y = 100.000
    XYZ[2] = float(XYZ[2]) / 108.883        # ref_Z = 108.883

    num = 0
    for value in XYZ:

        if value > 0.008856:
            value = value ** (0.3333333333333333)
        else:
            value = (7.787 * value) + (16 / 116)

        XYZ[num] = value
        num = num + 1

    Lab = [0, 0, 0]

    L = (116 * XYZ[1]) - 16
    a = 500 * (XYZ[0] - XYZ[1])
    b = 200 * (XYZ[1] - XYZ[2])

    Lab[0] = round(L, 4)
    Lab[1] = round(a, 4)
    Lab[2] = round(b, 4)

    return Lab

In [ ]:
#Algoritmo para convertir RGB a HSV, esto utilizando el formateo de arreglos (Mucho más rápido que hacer uno por uno)
import numpy as np

def rgb2hsv(rgb):
    rgb = rgb.astype('float')
    maxv = np.amax(rgb, axis=1)
    maxc = np.argmax(rgb, axis=1)
    minv = np.amin(rgb, axis=1)
    minc = np.argmin(rgb, axis=1)

    hsv = np.zeros(rgb.shape, dtype='float')
    hsv[maxc == minc, 0] = np.zeros(hsv[maxc == minc, 0].shape)
    hsv[maxc == 0, 0] = (((rgb[..., 1] - rgb[..., 2]) * 60.0 / (maxv - minv + np.spacing(1))) % 360.0)[maxc == 0]
    hsv[maxc == 1, 0] = (((rgb[..., 2] - rgb[..., 0]) * 60.0 / (maxv - minv + np.spacing(1))) + 120.0)[maxc == 1]
    hsv[maxc == 2, 0] = (((rgb[..., 0] - rgb[..., 1]) * 60.0 / (maxv - minv + np.spacing(1))) + 240.0)[maxc == 2]
    hsv[maxv == 0, 1] = np.zeros(hsv[maxv == 0, 1].shape)
    hsv[maxv != 0, 1] = (1 - minv / (maxv + np.spacing(1)))[maxv != 0]
    hsv[..., 2] = maxv

    return hsv

In [ ]:
#1. Obtener RGB's (Listo)
#2. Dividir cada imagen en 25 (Listo)
#3. Extracción de atributos (RGB, HSB, SIEL*A*B) || (Avg's values, Mediana, Moda, Desviación)

#Utilizado para guardar nuestros datos con el formato: Media Mediana Moda DesviacionEstandar de cada canal + Clasificacion 
import numpy as np
from scipy import stats
from IPython.display import clear_output 

with open('prueba.txt', 'w') as f:
    f.write('')

linea = ''
for z in range(0, len(data)):
    if (data[z] == []):
        continue;
    arr = np.array(data[z])
    hsv = rgb2hsv(arr)
    lab = []
    linea = ''
    print(z)

    for x in data[z]:
        lab.append(rgb2lab2(x))
    arr_lab = np.array(lab)    

    media =  np.mean(arr, 0)
    mediana = np.median(arr, 0)
    moda = stats.mode(arr, 0)
    desviacion = np.std(arr, 0) 
    linea += str(media) + str(mediana) + str(moda[0][0]) + str(desviacion)

    media =  np.mean(hsv, 0)
    mediana = np.median(hsv, 0)
    moda = stats.mode(hsv, 0)
    desviacion = np.std(hsv, 0) 
    linea += str(media) + str(mediana) + str(moda[0][0]) + str(desviacion)

    media =  np.mean(lab, 0)
    mediana = np.median(lab, 0)
    moda = stats.mode(lab, 0)
    desviacion = np.std(lab, 0) 
    linea += str(media) + str(mediana) + str(moda[0][0]) + str(desviacion)
    
    linea += str(clasificacion_1[z + 2500])

    with open('prueba.txt', 'a') as f:
        f.write(linea)
        f.write('\n')

    
    